# Part 2: Modeling & Evaluation

## Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
from keras.models            import Sequential
from keras.layers            import Dense, Dropout
from keras.optimizers        import Adam
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics         import roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.metrics         import precision_score, recall_score, f1_score
from sklearn.pipeline        import Pipeline
from IPython.core.display    import display, HTML
from IPython.display         import display_html
sns.set(style = "white", palette = "deep")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

## Table Of Contents

- [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)
    
    
- [Feature Engineering](#Feature-Engineering)
    - [Data Manipulation](#Data-Manipulation)
    - [Interaction Columns](#Interaction-Columns)
    
    
- [Establishing The Baseline](#Establishing-The-Baseline)
    
    
- [Modeling](#Modeling)
    - [Subset Definition](#Subset-Definition)
    - [Defining X & y Variables](#Defining-X-&-y-Variables)
    - [Train-Test Split](#Train-Test-Split)
    - [Evaluation Functions](#Evaluation-Functions)

## Reading In The Data

In [ ]:
pulsar = pd.read_csv("../Data/pulsar_cleaned.csv")

### Overview

In [ ]:
# Checking the head of the data

pulsar.head()

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {pulsar.shape}")

In [ ]:
# Summary of column data types

pulsar.dtypes.value_counts()

In [ ]:
# Checking for null values

pulsar.isnull().sum()

## Feature Engineering

### Data Manipulation

While visualizing the data, we noticed that there are two columns that appeared to have a close to normal distribution.  As a result, we decided to square the values to transform them.

In [ ]:
# Squaring the `mean_ip` column

pulsar["mean_ip_squared"] = pulsar["mean_ip"].apply(lambda x: x**2)

# Squaring the `sd_ip` column

pulsar["sd_ip_squared"]   = pulsar["sd_ip"].apply(lambda x: x**2)

In [ ]:
# Making sure new columns were added

pulsar.columns

### Interaction Columns

Based off of the heat map in the previous notebook, we noticed that there are some columns with very high correlations.  We felt that creating interaction columns, we would be emphasizing the correlation while also reducing the number of features.


The columns in particular are:


| Column 1   | Column 2   | Correlation |
|:-----------|:-----------|:-----------:|
| mean_ip    | sd_ip      | 0.55        |
| ex_kurt_ip | skew_ip    | 0.95        |
| mean_dmsnr | sd_dmsnr   | 0.80        |
| ex_kurt_ip | skew_dmsnr | 0.92        |


We chose these columns in particular because they allows us to reduce the number of features that will end up going into the model.  However, we decided not to get ride of the eight original features: we will end up with sub-sets of the model we will model on.

In [ ]:
# Defining the `mean_ip` * `sd_ip` colum
# Defining the `ex_kurt_ip` * `skew_ip` colum

pulsar["mean_*_sd_ip"]     = pulsar["mean_ip"] * pulsar["sd_ip"]
pulsar["exkurt_*_skew_ip"] = pulsar["ex_kurt_ip"] * pulsar["skew_ip"]

# Defining the `mean_dmsnr` * `sd_dmsnr` colum
# Defining the `ex_kurt_dmsnr` * `skew_dmsnr` colum

pulsar["mean_*_sd_dmsnr"]     = pulsar["mean_dmsnr"] * pulsar["sd_dmsnr"]
pulsar["exkurt_*_skew_dmsnr"] = pulsar["ex_kurt_dmsnr"] * pulsar["skew_dmsnr"]

In [ ]:
# Checking to make sure columns were created

pulsar.columns

## Establishing The Baseline

We need to know what our baseline accuracy is because that will give us an accuracy score to beat: if our accuracy is less than the baseline it means that our model is worse than guessing the category of a star.

In [ ]:
# Calculating the percentages of each class

round(pulsar["target_class"].value_counts(normalize = True)*100, 2)

In [ ]:
# Plotting the distribution of the classes

tick_labels = ["Non-Pulsar", "Pulsar"]

# Setting the figure size
plt.figure(figsize = (10,5))

# Plotting the graph
sns.countplot(pulsar["target_class"])

# Setting graph parameters
plt.title("Pulsar Or Not A Pulsar?", size = 18)
plt.xlabel("Star Type", size = 16)
plt.ylabel("Number Of Stars", size = 16)

# Making sure the only ticks are 0 and 1
plt.xticks(np.arange(0,2,1),
           labels = tick_labels,
           size   = 14)
plt.yticks(size = 14);

The data is _extremely_ imbalanced: this will make it difficult to model because the negative class (non-pulsar) is so much less frequent than the positive class (pulsar).

## Modeling

We decided that because of the extreme imbalance of the classes a feed forward neural network is the best approach to predicting the presence of a pulsar.

### Subset Definition

Because of the way we set up the interaction columns and manipulated the data, we decided that we are best off with three subsets: the original features, original features with squared columns, and the interaction columns.  We have to define the subsets before we define our X and y variables.

In [ ]:
pulsar.columns

In [ ]:
# List of the original features

original_features = ["mean_ip", "sd_ip", "ex_kurt_ip", "skew_ip",
                     "mean_dmsnr", "sd_dmsnr", "ex_kurt_dmsnr", "skew_dmsnr",
                     "target_class"]

# List of the original features with `mean_ip` and `sd_ip` squared

manipulated_features = ["mean_ip_squared", "sd_ip_squared", "ex_kurt_ip", "skew_ip",
                        "mean_dmsnr", "sd_dmsnr", "ex_kurt_dmsnr", "skew_dmsnr",
                        "target_class"]

# List of the interaction features

interaction_features = ["mean_*_sd_ip", "exkurt_*_skew_ip", "mean_*_sd_dmsnr",
                        "exkurt_*_skew_dmsnr", "target_class"]

In [ ]:
# Defining a subset with the original features

pulsar_og = pulsar[original_features]

# Defining a subset with the original/squared features

pulsar_sq = pulsar[manipulated_features]

# Defining a subset with the interaction features

pulsar_if = pulsar[interaction_features]

### Defining X & y Variables

From this point on `X` and `y` will refer to the original features, `_sq` will refer to the dataframe with the squared features, and `_if` will refer to the dataframe with interaction features.

In [ ]:
# X and y for the original

X = pulsar_og.drop("target_class", axis = 1)
y = pulsar_og["target_class"]

# X and y for the original/squared features

X_sq = pulsar_sq.drop("target_class", axis = 1)
y_sq = pulsar_sq["target_class"]

# X and y for the interaction features

X_if = pulsar_int.drop("target_class", axis = 1)
y_if = pulsar_int["target_class"]

### Train-Test Split

The purpose of the train-test split is to split up our data so that we can reserve an unseen portion of it to test our model on.  Additionally, we will be setting a random state for reproducability and we will stratify on `y` so that the distribution of classes is preserved.

In [ ]:
# For the original data

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 42,
                                                    stratify     = y)

# For the original data with squared features

X_sq_train, X_sq_test, y_sq_train, y_sq_test = train_test_split(X_sq,
                                                                y_sq,
                                                                random_state = 42,
                                                                stratify     = y_sq )

# For the dataframe with interaction features

X_if_train, X_if_test, y_if_train, y_if_test = train_test_split(X_if,
                                                                y_if,
                                                                random_state = 42,
                                                                stratify     = y_if)

### Evaluation Functions

A confusion matrix gives us a summary of how our model classified the test data: it compares the true and predicted y values so that we can see how the model performed on each class.


Each confusion matrix is set up the same way:

|                     | Predicted Positive | Predicted Negative |
|:--------------------|:------------------:|:------------------:|
| **Actual Positive** | True Positive      | False Negative     |
| **Actual Negative** | False Positive     | True Negative      |

In [ ]:
# We converted the confusion matrix to a dataframe to make it easier to read

def create_confusion_matrix(y, y_preds):
    cm     = confusion_matrix(y, y_preds)
    matrix = pd.DataFrame(cm, 
                          columns = ["Predicted r/Cooking", "Predicted r/AskCulinary"], 
                          index   = ["Actual r/Cooking", "Actual r/AskCulinary"])
    return matrix

We will generate two scores based off of the confusion matrix: specificity and sensitivity; we will go into what these scores measure in a few cells.

In [ ]:
# Calculating specificity from a confusion matrix

def specificity(y, y_pred):
    cm          = confusion_matrix(y, y_pred)  
    specificity = cm[1,1] / (cm[1,1] + cm[1,0])
    return specificity

# Calculating sensitivity from a confusion matrix

def sensitivity(y, y_pred):
    cm          = confusion_matrix(y, y_pred)
    sensitivity = cm[0,0] / (cm[0,0] + cm[0,1])
    return sensitivity

In addition to metric scores, we will also calculate an ROC-AUC score.  The ROC (receiver operating characteristic) shows us a binary classification model's ability to distinguish between two classes; we will plot this curve for the best model as determined by metric scores.

These images from [GreyAtom](https://medium.com/greyatom/lets-learn-about-auc-roc-curve-4a94b4d88152) illustrates the AUC-ROC well:

<img src = "../Images/ROC_AUC 0.8 0.9.png" alt = "high auc_roc scores" height = "350" width = "350">

<img src = "../Images/ROC_AUC 0.5 0.7.png" alt = "low auc_roc scores" height = "350" width = "350">